In [33]:
from keras.applications import vgg16
# from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input
import tensorflow as tf

from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

#IMPORTS put all here
import random
import matplotlib.pyplot as plt

from datetime import datetime

#tf imports for reading file
import numpy as np
import os
import PIL


import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow import keras


from tensorflow.keras.regularizers import l2

#extras
print(tf.__version__)
import pathlib
import shutil

2.9.1


In [34]:
# parameters setup

# imgs_path = "../input/style/"
imgs_model_width, imgs_model_height = 224, 224

nb_closest_images = 5 # number of most similar images to retrieve

In [35]:
dest_path='../content/new/spectrograms6secnew'

data_dir=pathlib.Path(dest_path)
data_dir

WindowsPath('../content/new/spectrograms6secnew')

In [36]:
#adding dataset
batch_size=64
image_height=75
image_width=150

In [37]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

Found 29990 files belonging to 10 classes.
Using 23992 files for training.


In [38]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

Found 29990 files belonging to 10 classes.
Using 5998 files for validation.


In [39]:
class_names = train_ds.class_names
print(class_names)

['bhojpuri_pop', 'carnatic', 'classic_bollywood', 'desi_pop', 'ghazal', 'hindustani_classical', 'indian_indie', 'punjabi_hip_hop', 'sufi', 'tamil_pop']


In [78]:
num_classes = len(class_names)
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Input(shape=(75, 150, 3)))
model.add(tf.keras.layers.Conv2D(16, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Compile model
model.compile('adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

epochs=5

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )

Epoch 1/5
375/375 [==============================] - 263s 678ms/step - loss: 2.0133 - accuracy: 0.3692 - val_loss: 3.1283 - val_accuracy: 0.1664
Epoch 2/5
375/375 [==============================] - 239s 628ms/step - loss: 1.4864 - accuracy: 0.4897 - val_loss: 1.9156 - val_accuracy: 0.3740
Epoch 3/5
375/375 [==============================] - 246s 643ms/step - loss: 1.3301 - accuracy: 0.5364 - val_loss: 1.6213 - val_accuracy: 0.4533
Epoch 4/5
375/375 [==============================] - 239s 627ms/step - loss: 1.2478 - accuracy: 0.5646 - val_loss: 1.3911 - val_accuracy: 0.5165
Epoch 5/5
375/375 [==============================] - 236s 618ms/step - loss: 1.1953 - accuracy: 0.5823 - val_loss: 1.6869 - val_accuracy: 0.4520


In [40]:
from tensorflow.keras.models import load_model

model = load_model(".\models\models")

OSError: SavedModel file does not exist at: .\models\models\{saved_model.pbtxt|saved_model.pb}

In [12]:
import cv2
import numpy as np
cnt = 0
path = os.path.join(dest_path,'sufi')
for images in os.listdir(path):
    print(os.path.join(path,images))
    img = cv2.imread(os.path.join(path,images))
    img = tf.image.resize(img,(75,150))
    yhat = model.predict(np.expand_dims(img/255.0,axis=0))
    print(yhat)
    print(yhat.argmax())
    cnt+=1
    if cnt == 5:
        break

../content/new/spectrograms6secnew\sufi\A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina1.png
1/1 [==============================] - 0s 25ms/step
[[8.2908067e-20 7.4943743e-21 9.9994600e-01 8.3869644e-21 1.9165328e-19
  5.4051899e-05 9.2827226e-18 4.7357246e-27 1.2696037e-36 5.8324525e-17]]
2
../content/new/spectrograms6secnew\sufi\A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina10.png
1/1 [==============================] - 0s 26ms/step
[[6.8197513e-20 7.5745607e-21 9.9993420e-01 7.4105358e-21 1.9626251e-19
  6.5766610e-05 8.6627499e-18 4.5716893e-27 1.1676563e-36 5.0065844e-17]]
2
../content/new/spectrograms6secnew\sufi\A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina12.png
1/1 [==============================] - 0s 34ms/step
[[6.2310423e-20 7.7554822e-21 9.9993730e-01 6.9970925e-21 1.7383173e-19
  6.2718151e-05 8.0501457e-18 4.2413322e-27 1.0319195e-36 4.8569706e-17]]
2
../content/new/spectrograms6secnew\sufi\A.R. Rahman, Chinmayi, Murtuza Khan, Qadir 

In [41]:
from PIL import Image
from scipy import spatial

test_image1 = Image.open("../content/new/spectrograms6secnew\sufi\A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina14.png")
test_image2 = Image.open("../content/new/spectrograms6secnew\sufi\A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina10.png")

img_1 = test_image1.resize((100,200))
img_2 = test_image2.resize((100,200))

arr_1 = np.array(img_1)
arr_2 = np.array(img_2)

arr_1 = arr_1.flatten()
arr_2 = arr_2.flatten()

arr_1 = arr_1/255
arr_2 = arr_2/255

similarity = -1 * (spatial.distance.cosine(arr_1,arr_2) - 1)
print(similarity)


0.9941433396315041


In [48]:
def get_image_array(image):
    img = image.resize((100,200))
    img_arr = np.array(img)
    img_arr = img_arr.flatten()
    img_arr = img_arr/255
    return img_arr

In [52]:
compared_image = get_image_array(test_image1)
similarity_images = []
cnt = 0
path = os.path.join(dest_path,'sufi')
for images in os.listdir(path):
    image_obj = Image.open(os.path.join(path,images))
    
    image2 = get_image_array(image_obj)
    similarity = -1 * (spatial.distance.cosine(compared_image,image2) - 1)
    image_name = images.split('\\')[-1]
    similarity_images.append([image_name,similarity])
    cnt+=1
    if cnt == 10:
        break

print(similarity_images)

[['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina1.png', 0.9884426057340217], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina10.png', 0.9941433396315041], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina12.png', 0.9946501562544591], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina13.png', 0.9947671280478219], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina14.png', 1], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina16.png', 0.9943340744466141], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina18.png', 0.9942302193022037], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina2.png', 0.9937414928944845], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina21.png', 0.9937744353109781], ['A.R. Rahman, Chinmayi, Murtuza Khan, Qadir Khan - Tere Bina26.png', 0.9934529220706868]]
